<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/Pspell_check%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_C_inc.csv	   df_K_inc.csv     spell_check
'Colab Notebooks'	'# df_G_inc.csv'   df_P_inc.csv
 C_ori_model.pt		 df_G_inc.csv	  'My Drive'
'답안 작성용 파일.csv'	 df_J_inc.csv	   Q_inc_model.pt


In [3]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [4]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [5]:
df=df[(df['digit_1'] =='P')] #조건에 맞는 데이터 출력

In [6]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000016,P,85,856,태권도장에서,학생및일반대사으로,태권도
1,id_0000028,P,85,856,사무실에서,직원들대상,교육관
2,id_0000054,P,85,855,학원,"학생, 일반인대상으로",중국어교육
3,id_0000140,P,85,856,사업장에서,일반고객대상으로,요가
4,id_0000143,P,85,856,학원에서,일반인대상으로,캘리그라피
...,...,...,...,...,...,...,...
46605,id_0999956,P,85,855,학원에서,학생을 대상으로,영어
46606,id_0999965,P,85,856,학원에서,일반인 대상으로,"캘리그라프,팝아트 등"
46607,id_0999966,P,85,855,학원에서,중고생을 대상으로,수학 교습
46608,id_0999990,P,85,855,공부방에서,학생대상,교과목


In [7]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000016,P,85,856,태권도장에서 학생및일반대사으로 태권도
1,id_0000028,P,85,856,사무실에서 직원들대상 교육관
2,id_0000054,P,85,855,"학원 학생, 일반인대상으로 중국어교육"
3,id_0000140,P,85,856,사업장에서 일반고객대상으로 요가
4,id_0000143,P,85,856,학원에서 일반인대상으로 캘리그라피
...,...,...,...,...,...
46605,id_0999956,P,85,855,학원에서 학생을 대상으로 영어
46606,id_0999965,P,85,856,"학원에서 일반인 대상으로 캘리그라프,팝아트 등"
46607,id_0999966,P,85,855,학원에서 중고생을 대상으로 수학 교습
46608,id_0999990,P,85,855,공부방에서 학생대상 교과목


In [8]:
df['digit_3'].value_counts()

856    23521
855    16862
851     3544
852     1333
857      917
853      366
854       67
Name: digit_3, dtype: int64

## 맞춤법 교정

In [9]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-1ydmetcg
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-1ydmetcg
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=0254742e19037d62b520707b3aea691df7e9030db11899d95cb8303ba3748b7d
  Stored in directory: /tmp/pip-ephem-wheel-cache-xmwkrrxn/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [10]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [11]:
df.to_csv('df_P_spellcheck.csv', index=False, encoding='utf-8-sig')